In [ ]:
# Для нейросети
from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.models import Sequential # Полносвязная модель
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Activation # Слои для сети
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами
from tensorflow.keras.preprocessing.sequence import pad_sequences # Метод для работы с последовательностями

import numpy as np # Для работы с данными 
import matplotlib.pyplot as plt # Для вывода графиков
import json

import os # Для работы с файлами
from google.colab import files # Для работы с файлами 
from google.colab import drive # Для работы с Google Drive

%matplotlib inline

In [ ]:
drive.mount('drive', force_remount=True)

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
dataset_dir = '/content/drive/MyDrive/Future_Foundation/Up_Great_Сколково/Satellites'

Все тексты true и false объединены в два больших текста;
Датасет - список из двух элементов-строк

In [ ]:
# создаём два списка, в пустые элементы которых будем добавлять тексты
# в первом элементе будет храниться текст с ответом False, во втором - True
trainText = ['', '']
testText = ['', '']

# открываем файл с id эссе и ответами
with open('/content/drive/MyDrive/Future_Foundation/Up_Great_Сколково/Satellites/train/train_standart.json', 'r') as f_list:
  data = json.load(f_list)

  # проходимся по каждому "блоку" с эссе
  for i in range(len(data)):
    elem = data[i]

    # ввыделяем на тестовую выборку 10 процентов
    if i < len(data)//10:

      # с помощью форматированной строки открываем файл в названии которого id, который указан в блоке
      with open(f'/content/drive/MyDrive/Future_Foundation/Up_Great_Сколково/Satellites/train/essays/{elem["id"]}.json', 'r') as essay:
        file = json.load(essay)
        text = file['text']
        
        if elem['answer'] == False:
          testText[0] += text
          testText[0] += ' '
        else:
          testText[1] += text
          testText[1] += ' '
      continue

    with open(f'/content/drive/MyDrive/Future_Foundation/Up_Great_Сколково/Satellites/train/essays/{elem["id"]}.json', 'r') as essay:
      file = json.load(essay)
      text = file['text']
      if elem['answer'] == False:
        trainText[0] += text
        trainText[0] += ' '
      else:
        trainText[1] += text
        trainText[1] += ' '

In [ ]:
numWords = 20000 # Количество слов/индексов, которое мы будем учитывать при обучении

# настраиваем токенайзер, все слова приводятся в нижний регистр, специальные символы опускаются
tokenizer = Tokenizer(num_words=numWords,
                      filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
                      lower=True,
                      split=' ',
                      oov_token='unknown',
                      char_level=False)

# Применяем токенайзер
tokenizer.fit_on_texts(trainText) # словарь частотности # НА САМОМ ДЕЛЕ ЗДЕСЬ МАССИВ ЭССЕ
items = list(tokenizer.word_index.items()) # индексы слов

In [ ]:
print("Самые часто встречающиеся слова:")
print(items[:10])
print()
print("Самые редко встречающиеся слова:")
print(items[-10:])
print()
print("Размер словаря:", len(items))

Самые часто встречающиеся слова:
[('unknown', 1), ('в', 2), ('и', 3), ('на', 4), ('что', 5), ('не', 6), ('с', 7), ('к', 8), ('как', 9), ('а', 10)]

Самые редко встречающиеся слова:
[('дозволено', 49697), ('вошью', 49698), ('мешавшей', 49699), ('старухину', 49700), ('жуткого', 49701), ('сломает', 49702), ('восставала', 49703), ('пришедший', 49704), ('пролитию', 49705), ('несущей', 49706)]

Размер словаря: 49706


In [ ]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
trainWordIndexes = tokenizer.texts_to_sequences(trainText)
testWordIndexes = tokenizer.texts_to_sequences(testText)

In [ ]:
# классы, на которые делятся тексты
labels = ['false', 'true']
labelsNum = len(labels)

In [ ]:
print("Статистика по обучающим текстам:")

symbolsTrainText = 0
wordsTrainText = 0

for i in range(labelsNum):
  print(labels[i], " "*(10-len(labels[i])), len(trainText[i]), "символов, ", len(trainWordIndexes[i]), "слов")
  symbolsTrainText += len(trainText[i])
  wordsTrainText += len(trainWordIndexes[i])

print('----')
print("В сумме ", symbolsTrainText, " символов, ", wordsTrainText, " слов \n")
print()

print("Статистика по тестовым текстам:")

symbolsTestText = 0
wordsTestText = 0

for i in range(labelsNum):
  print(labels[i], ' '*(10-len(labels[i])), len(testText[i]), "символов, ", len(testWordIndexes[i]), "слов")
  symbolsTestText += len(testText[i])
  wordsTestText += len(testWordIndexes[i])
print('----')
print("В сумме ", symbolsTestText, " символов, ", wordsTestText, " слов")

Статистика по обучающим текстам:
false       2510796 символов,  346573 слов
true        423041 символов,  57646 слов
----
В сумме  2933837  символов,  404219  слов 


Статистика по тестовым текстам:
false       76757 символов,  10189 слов
true        231715 символов,  33470 слов
----
В сумме  308472  символов,  43659  слов


trainWordIndexes
testWordIndexes

их имеет смысл сохранить как отдельные датасеты.
эти датасеты можно будет потом открывать в колабах с разными структурами нейросетей 
обрабатывать и менять их как необходимо.
Это позволит не тратить время и ресурсы виртуальной среды на новую сборку датасета.

In [ ]:
# Преобразование в np массивы
trainWordIndexes = np.array(trainWordIndexes)
testWordIndexes = np.array(testWordIndexes)

# Проверка преобразования
print(type(trainWordIndexes))
print(trainWordIndexes)
print(type(testWordIndexes))
print(testWordIndexes)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Сохренение списков trainWordIndexes и testWordIndexes как файлы

# # Функция сохранения файлов
def saveList(myList,filename):
    # the filename should mention the extension 'npy'
    np.save(filename,myList)
    print("Saved successfully!")

# # Сохранение списков в виде файлов на наш гугл диск
saveList(trainWordIndexes, '/content/drive/MyDrive/Future_Foundation/Up_Great_Сколково/Colabs_to_create_datasets/trainWordIndexes.npy')
saveList(testWordIndexes, '/content/drive/MyDrive/Future_Foundation/Up_Great_Сколково/Colabs_to_create_datasets/testWordIndexes.npy')

# # Как загрузить такой тип файла и преобразовать в список обратно - смотри колаб Глеба 

Saved successfully!
Saved successfully!


In [ ]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText


# Формирование обучающей и проверочной выборки для каждого класса
# wordIndexes - массив индексов
# xLen - размер окна
# step - шаг окна

def createSetsMultiClasses(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wI in wordIndexes:      # Для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wI, xLen, step))

  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [ ]:
# ЭТИ ПАРАМЕТРЫ МОЖНО И НУЖНО МЕНЯТЬ
# Задаём базовые параметры
xLen = 50 # Размер окна (количество слов в векторе)
step = 6 # Шаг разбиения текста на векторы

In [ ]:
xTrainId, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTestId, yTest = createSetsMultiClasses(testWordIndexes, xLen, xLen)

# для каждого "окна" класса мы в соответствие ставим значение true или false => поэтому длины x и y одинаковые
print("Размерности тренировочного набора")
print(xTrainId.shape)
print(yTrain.shape)
print()
print("Размерности тестового набора")
print(xTestId.shape)
print(yTest.shape)

Размерности тренировочного набора
(67354, 50)
(67354, 2)

Размерности тестового набора
(872, 50)
(872, 2)


In [ ]:
print(xTrainId[0])

[ 5748    69   652    16  8497  1554   223   249   381  3499     3  6461
    30   124   255   303   336 16379     2   313   233  2400  1258   120
    29   391    14     1    45 12534  4694     2  1324  8498    27  3733
   846   298 16380 12535 16381   820    85   194   164  5168     2 16382
  7346    66]


In [ ]:
# xTrainIdList = xTrainId.tolist()
# xTestIdList = xTestId.tolist()

# print(type(xTrainIdList))
# print(type(xTestIdList))

# # Сохранение списков в файл, для формирования датасета в виде 


# Для модели BOW - преобразовываем в Bag Of Words
xTrain = tokenizer.sequences_to_matrix(xTrainId.tolist())
xTest  = tokenizer.sequences_to_matrix(xTestId.tolist())

print(type(xTrain))
print(type(xTest))


# Для остальных моделей - НЕ преобразовываем в Bag Of Words

# xTrain = xTrainId
# xTest = xTestId

print("Размерность обучайющей выборки")
print(xTrain.shape)
print(xTrain[0][0:100]) # Фрагмент набора слов в виде Bag of Words

print()

print("Размерность тестовой выборки")
print(xTest.shape)
print(xTest[0][0:100]) # Фрагмент набора слов в виде Bag of Words

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Размерность обучайющей выборки
(67354, 20000)
[0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]

Размерность тестовой выборки
(872, 20000)
[0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1.]
